In [ ]:
!pip install 'kaggle-environments>=0.1.6'

In [ ]:
import random
import math
import copy
import numpy as np
import gym
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam
from kaggle_environments import evaluate, make
from gym.spaces import Discrete

import matplotlib.pyplot as plt

### Short notes on gym.Env

Attributes:
+ action_space: The Space object corresponding to valid actions
+ observation_space: The Space object corresponding to valid observations
+ reward_range: A tuple corresponding to the min and max possible rewards

Main methods:
+ step()
+ reset()
+ render()
+ close()
+ seed()

# Connect-X Environment

In [ ]:
class ConnectX(gym.Env):
    def __init__(self):
        super(ConnectX, self).__init__()
        self.env = make('connectx', debug=False)
        self.pair = [None, 'random']
        self.side = 0
        self.trainer = self.env.train(self.pair)
        
        config = self.env.configuration
        self.action_space = gym.spaces.Discrete(config.columns) 
        self.observation_space = gym.spaces.Discrete(config.columns * config.rows + 1)
    
    def step(self, action):
        obs, rew, done, info = self.trainer.step(action)
        obs = obs['board'] + [obs['mark']]
        return (obs, rew, done, info)
    
    def reset(self):
        obs = self.trainer.reset()
        return obs['board'] + [obs['mark']] # TODO: Maybe also append 'mark'.  see: self.action_space
    
    def render(self, **kwargs):
        return self.env.render(**kwargs)
    
    def change_opponent(self, agent2):
        self.pair = [None, agent2]
        
    def get_side(self):
        return self.side
    
    def switch_side(self):
        self.pair = self.pair[::-1]
        self.trainer = self.env.train(self.pair)
        self.side = 1 - self.side % 2

# DQN

In [ ]:
class Replay:
    def __init__(self, capacity=1e4):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
        
#     def load_replay(self):
#         raise NotImplementedError
    
    def add_experience(self, experience):
        """
        
        Parameters:
        experience ([np.array]): Experience which should be stored in the replay buffer
        """
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[int(self.position)] = experience
        self.position = (self.position + 1) % self.capacity
    
    def get_batch(self, batch_size):
        sample = random.sample(self.buffer, batch_size)
        
        obs_batch, act_batch, rew_batch, next_obs_batch, dones_batch = [], [], [], [], []
        
        for s in sample:
            obs_batch.append(s[0])
            act_batch.append(s[1])
            rew_batch.append(s[2])
            next_obs_batch.append(s[3])
            dones_batch.append(s[4])
        obs_batch = torch.tensor(obs_batch, dtype=torch.float32)
        act_batch = torch.tensor(act_batch, dtype=torch.int64)
        rew_batch = torch.tensor(rew_batch, dtype=torch.float32)
        next_obs_batch = torch.tensor(next_obs_batch, dtype=torch.float32)
        dones_batch = torch.tensor(dones_batch, dtype=torch.float32)
        
        # num_entries = len(sample[0])
        
        # batch = []
        # for i in range(num_entries):
        #     batch_i = []
        #     for s in sample:
        #         batch_i.append(s[i])
        #     batch.append(batch_i)
        
        # torch_batch = [torch.tensor(b) for b in batch]
        return (obs_batch, act_batch, rew_batch, next_obs_batch, dones_batch)
    
    def get_size(self):
        return len(self.buffer)
    
    
class QNet(nn.Module):
    def __init__(self, env, n_hidden_layers=2, n_neurons=64):
        super(QNet, self).__init__()
        
        self.action_space = env.action_space.n if isinstance(env.action_space, Discrete) \
            else env.action_space.shape[0]
        self.observation_space = env.observation_space.n if isinstance(env.observation_space, Discrete) \
            else env.observation_space.shape[0]
        
        sizes = [self.observation_space] + [n_neurons for _ in range(n_hidden_layers)] + [self.action_space]
        layers = []
        for i in range(n_hidden_layers + 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1]))
            if i  < n_hidden_layers - 2:
                layers.append(nn.ReLU())
        self.layers = nn.Sequential(*layers)
    
    def forward(self, obs):
        return self.layers(obs)
    
    #def get_action(self, obs, epsilon=0.9, deterministic=False):
    #    '''
    #    choose action based on epsilon-greedy.
    #    '''
    #    if random.random() < epsilon or deterministic:
    #        return self.forward(obs).detach().argmax().item()
    #    else:
    #        return random.randint(0, self.action_space - 1)
        

class DQN:
    def __init__(self, env, seed=0,
                 replay_capacity=1e5, batch_size=100,
                 gamma=0.99, tau=0.995, eps_start=0.9, eps_decay=200, eps_end=0.05,
                 lr=1e-3, n_hidden_layers=2, n_neurons=64,**args):
        
        self.env = env
        
        self.action_space = env.action_space.n if isinstance(env.action_space, Discrete) \
            else env.action_space.shape[0]
        self.observation_space = env.observation_space.n if isinstance(env.observation_space, Discrete) \
            else env.observation_space.shape[0]
        
        self.seed = seed
        # self.replay = Replay(replay_capacity)
        self.batch_size = batch_size
        self.gamma = gamma
        self.tau = tau
        
        self.eps = eps_start
        self.eps_end = eps_end
        self.eps_decay = eps_decay
        
        # initialize model and target model
        self.critic = QNet(self.env, n_hidden_layers=n_hidden_layers, n_neurons=n_neurons)
        self.target_critic = QNet(self.env, n_hidden_layers=n_hidden_layers, n_neurons=n_neurons)
        for target_params in self.target_critic.parameters():
            target_params.requires_grad = False
        # copy parameter from critic to target_critic
        self._soft_update(tau=0)
#         for target_params, params in zip(self.target_critic.parameters(), self.critic.parameters()):
#             target_params.data.copy_(params.data)
        self.optimizer = Adam(self.critic.parameters(), lr=lr)
    
        self.loss = nn.MSELoss()
    
    def train(self, replay):
        obs, act, rew, next_obs, dones = replay.get_batch(self.batch_size)       
        
        act = act.unsqueeze(1)
        y = self.critic(obs).gather(1, act).squeeze()
        
        next_state_action_values = torch.max(self.target_critic(next_obs), 1).values.squeeze()
        target = rew + self.gamma * (1 - dones) * next_state_action_values
        
        # optimize Critic
        self.optimizer.zero_grad()
        # loss = self.loss(y, target)
        loss = ((y - target)**2).mean()
        loss.backward()
        self.optimizer.step()
        
        # Soft update
        # self._soft_update()
        
        return loss.detach()
        
    def evaluate(self):
        raise NotImplementedError
        
    def get_action(self, obs, deterministic=False):
        '''
        choose action based on epsilon-greedy.
        '''
        
        if random.random() > self.eps or deterministic:
            # disallow all actions where a stone is already placed
            state_value = self.critic.forward(obs).detach()
            for i in range(self.env.action_space.n):
                if obs[i] != 0:
                    state_value[i] = -1e7
            return state_value.argmax().item()
        
        else:
            return random.choice([a for a in range(self.env.action_space.n) if obs[a] == 0])
    
    def _soft_update(self, tau=None):
        polyak = tau if tau is not None else self.tau
        with torch.no_grad():
            for target_p, p in zip(self.target_critic.parameters(), self.critic.parameters()):
                target_p.data.mul_(polyak)
                target_p.data.add_((1 - polyak) * p.data)
    
    def _update_eps(self):
        self.eps = self.eps_end * (1 - math.exp(-1/self.eps_decay)) + self.eps * math.exp(-1/self.eps_decay)

# Multi-Agent Reinforcement Learning

In [ ]:
def play_game(env, model, replay=None, side=None, render=False, deterministic=False, train=False):
    if side is None:
        if random.randint(0,1):
            env.switch_side()
    elif side == 0:
        if env.get_side() == 1:
            env.switch_side()
    elif side == 1: 
        if env.get_side() == 0:
            env.switch_side()
    # print('side: ', env.get_side())
    rewards = []
    cum_reward = 0
    done = False
    obs = env.reset()
    obs = torch.tensor(obs).float()
    # print(obs)
    while not done:
        action = model.get_action(obs, deterministic=deterministic)
        next_obs, rew, done, _ = env.step(action)
        next_obs = torch.tensor(next_obs).float()
        # print(obs[:-1].view(6, -1))
        # print(rew)
        
        if rew is None:
            rew = 0
        if done:
            if rew == 1:
                rew = 20
            elif rew == -1:
                rew = -20
            else:
                rew = 10
        else:
            rew = 0.5
        
        cum_reward += rew
        rewards.append(rew)
        
        # add experience to replay buffer
        if replay is not None:
            replay.add_experience((obs.numpy(), action, rew, next_obs.numpy(), done))
        if train:
            loss = model.train(replay)
        obs = next_obs
    if render:
        env.render(mode="ipython", width=500, height=450)
    
    if replay is None:
        return cum_reward
    else:
        if train:
            return replay, loss
        return replay
    
    
def train_model(env, model, replay, epochs=100, min_buffer_size=1000, **args):
    buffer_size = replay.get_size()
    rewards = []
    while replay.get_size() < min_buffer_size:
        replay = play_game(env, model, replay)
    for i in range(epochs):
        replay, loss = play_game(env, model, replay, train=True)
        # loss = model.train(replay)
        model._update_eps()
        if (i % 25) == 0:
            model._soft_update(tau=0)
        if ((i + 1) % 10) == 0:
            cum_rewards_array = []
            for _ in range(25):
                cum_reward = play_game(env, model, deterministic=True)
                cum_rewards_array.append(cum_reward)
            print('Episode: ', i + 1, '\tCritic loss: ', loss.item(), '\tMean cumulated reward: ', np.array(cum_rewards_array).mean())
            rewards.append(np.array(cum_rewards_array).mean())
    return rewards

# Initialize training/Hyperparameter setting

In [ ]:
hyperparameter = {'lr': 1e-4,
                  'replay_capacity': 1e6,
                  'batch_size': 128,
                  'gamma': 0.99,
                  'tau': 0.995,
                  'eps_start': 0.9,
                  'eps_decay': 200, 
                  'eps_end': 0.05,
                  'epochs': 10000,
                  'min_buffer_size': 1000,
                  'n_hidden_layers': 4,
                  'n_neurons': 64
                 }

# Train Agent

In [ ]:
env = ConnectX()
# env.change_opponent('negamax')
replay = Replay()
dqn = DQN(env, **hyperparameter)

rewards = train_model(env, dqn, replay, **hyperparameter)

print('Side selection: {}'.format(env.get_side()))
play_game(env, dqn, render=True, side=0, deterministic=True)

In [ ]:
plt.figure()
plt.plot(rewards)
plt.show()

In [ ]:
play_game(env, dqn, render=True, deterministic=True)
# print('Side', env.get_side(), env.pair)

In [ ]:
env_play = make("connectx", debug=True)